
---

# 📈 15. Monitoring & Logging 

## A) Quick metrics (built-in)

```bash
docker stats --no-stream          # CPU/RAM/IO per container (one shot)
docker top <name>                 # processes inside container
docker inspect <name>             # JSON: limits, health, mounts, IP
```

---

## B) Runtime logs (built-in)

```bash
docker logs -f --tail 100 <name>  # follow recent logs
```

**Rotate logs (per-container)**

```bash
docker run --log-driver json-file \
  --log-opt max-size=10m --log-opt max-file=3 myapp
```

> Use `--log-driver local` for smaller on-disk footprint.
> Linux raw files: `/var/lib/docker/containers/<id>/*-json.log`

---

## C) Prometheus + Grafana (metrics)

**cAdvisor (exports Docker/container metrics)**

```bash
docker run -d --name=cadvisor -p 8080:8080 \
  -v /:/rootfs:ro -v /var/run:/var/run:rw \
  -v /sys:/sys:ro -v /var/lib/docker/:/var/lib/docker:ro \
  gcr.io/cadvisor/cadvisor
```

**Prometheus scrape (snippet)**

```yaml
scrape_configs:
  - job_name: cadvisor
    static_configs: [{ targets: ["localhost:8080"] }]
```

Then add Prometheus as a **Grafana** data source and import a Docker/cAdvisor dashboard.

---

## D) Centralized logs (ELK/EFK)

**Option 1: logging driver → collector**

```bash
docker run --log-driver fluentd \
  --log-opt fluentd-address=localhost:24224 myapp
```

**Option 2: file shipper**

* Keep default logs; use **Filebeat/Fluent Bit** to ship
  `/var/lib/docker/containers/*/*.log` → **Elasticsearch**; view in **Kibana**.

---

## E) Events & health

```bash
docker events                      # live Docker daemon events
docker inspect -f '{{.State.Health.Status}}' <name>  # healthy?
```

Add probe (image has HTTP endpoint):

```dockerfile
HEALTHCHECK CMD curl -fsS http://localhost/health || exit 1
```

---

## F) Best practices (1-liners)

* Use **`docker stats`** for quick checks; **Prometheus+cAdvisor** for dashboards.
* Prefer **`local`/`json-file`** drivers with **rotation**.
* Don’t write secrets to logs; scrub at app/collector.
* Name containers consistently; add labels for filtering.
* Keep metrics/logs in **separate** stores (Prometheus ≠ logs).
